### Problem setup

In [21]:
import time
import random
import warnings
import pandas as pd

warnings.filterwarnings("ignore")

In [22]:
defects = pd.read_csv('defects.csv')
defects = defects.dropna()
defects = defects.drop_duplicates()
defects.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   x       500 non-null    float64
 1   class   500 non-null    object 
dtypes: float64(1), object(1)
memory usage: 7.9+ KB


In [23]:
#on trie les défauts par position
defects = defects.sort_values(by=['x'])
defects['x'] = defects['x'].astype(int)
defects.head(20)

,x,class
479,0,a
141,0,a
142,1,a
493,2,b
125,2,a
403,3,a
61,3,b
72,8,c
112,9,c
33,9,c


In [24]:
import re

def display_defects():
    defects_count = defects.groupby(['x', 'class']).size().unstack(fill_value=0)
    defects_count = defects_count.reindex(range(500), fill_value=0)
    defects_count = defects_count.replace(0, '-')
    defects_count = defects_count.transpose()

    for i in range(3):
        line = defects_count.iloc[i].to_string()

        # Use regular expression to match and replace the pattern
        output_string = re.sub(r'(.)(?:\n\d+)', r'\1', line)
        res = output_string[1:].replace(" ", "")

        if i == 0:
            print("Class A")
        elif i == 1:
            print("Class B")
        else:
            print("Class C")
            
        for i in range(0, len(res), 100):
            print(res[i:i+100], end="\n")
        print("_"*100+"\n", end="\n")

display_defects()

Class A
2111--------1-1-1-----12-11--1--1---1-1-----1-1-1--1-1-1--11---------1---1--12-----1-2------12-1----
---1---11---1---1221----13--------------1--21---------1-1-11---3--1--1-11---------------311---------
---1-2------1-1-1-1----1---1--1-----------3---1----1-1----------------1----211-12---1---11----1-----
-11-1--------12--------------11-1-1-----1-------1-------1-----1-11-121-1-1------1----1-----11-------
-----11-11----1-1---------2--111-----12-1--1-12--1---1--11----1-1----1-----1-12--1112---11------11--
____________________________________________________________________________________________________

Class B
--11------1--------1-11-------1--11111------1-1-1--1--1----1--------1---2---------21------------1---
-------1--------11----11---1-1---------1-1----1-1----11-13---1-1--3-2--1-1--1--2----3111--111-------
1--------1---11-1----1-----1------11-----31--1--1--1-----111-------1-11-1--2-----1--11---1-1112-1---
-11-1-----1----1-----1--1-------2-11--1-----1--1---111-1-11-1--1-------1--

In [25]:
class Biscuit:

    def __init__(self, length, value, defects, id=None):
        self.length = length
        self.value = value
        self.defects = defects
        self.id = id

biscuit_0 = Biscuit(length=4, value=6, defects={"a":4, "b":2, "c":3}, id=0)
biscuit_1 = Biscuit(length=8, value=12, defects={"a":5, "b":4, "c":4}, id=1)
biscuit_2 = Biscuit(length=2, value=1, defects={"a":1, "b":2, "c":1}, id=2)
biscuit_3 = Biscuit(length=5, value=8, defects={"a":2, "b":3, "c":2}, id=3)

In [26]:
class Roll(Biscuit):
    
    def __init__(self, biscuits_and_positions, length):
        self.biscuits_and_positions = biscuits_and_positions
        self.defects = self.get_defects()
        self.length = length 
        self.score = self.get_score()
    
    def get_defects(self):
        defects_roll = {}
        for i in range(len(defects)):
            defects_roll[f"{defects.iloc[i]['x']}"] = defects.iloc[i]['class']
        return defects_roll
    
    def get_score(self):
        score = 0
        for biscuit in self.biscuits_and_positions:
            score += biscuit[0].value
        return score
    
    def can_be_added(self, biscuit, index, print_enabled=False):
        res = False
        biscuit = (biscuit, index)
        

        #on vérifie que le biscuit est bien dans le roll
        if biscuit[1]>=0 and biscuit[1]+biscuit[0].length-1<=self.length-1:
            res = True

            #on vérifie que le biscuit n'a pas trop de defauts
            defects_count = defects.groupby(['x', 'class']).size().unstack(fill_value=0)
            defects_count = defects_count.reindex(range(500), fill_value=0)
            defects_count = defects_count.transpose()

            start_index = biscuit[1]
            end_index = biscuit[1]+biscuit[0].length-1

            selected_range = defects_count.loc[:, start_index:end_index]
            count_defect_a = selected_range.loc['a'].sum()
            count_defect_b = selected_range.loc['b'].sum()
            count_defect_c = selected_range.loc['c'].sum()

            if count_defect_a > biscuit[0].defects['a']:
                res = False
                if print_enabled:
                    print("Too many defect a")
            elif count_defect_b > biscuit[0].defects['b']:
                res = False
                if print_enabled:
                    print("Too many defect b")
            elif count_defect_c > biscuit[0].defects['c']:
                res = False
                if print_enabled:
                    print("Too many defect c")

            #on vérifie que le biscuit ne chevauche pas un autre biscuit
            for biscuit_2 in self.biscuits_and_positions:
                start_1, end_1 = biscuit[1], biscuit[1] + biscuit[0].length -1
                start_2, end_2 = biscuit_2[1], biscuit_2[1] + biscuit_2[0].length -1

                if start_1 <= end_2 and end_1 >= start_2:
                    res = False
                    if print_enabled:
                        print("Biscuit overlap")
                    break
        if res and print_enabled:
            print("Biscuit can be added")
        return res
    
    def add_biscuit(self, biscuit, index, print_enabled=False):
        if print_enabled:
            print(f"Trying to add (biscuit_{biscuit.id}, {index}):")
        if self.can_be_added(biscuit, index, print_enabled=print_enabled):
            self.biscuits_and_positions.append((biscuit, index))
            self.biscuits_and_positions = sorted(self.biscuits_and_positions, key=lambda x: x[1])

            #update score 
            self.score += biscuit.value
        else:
            if print_enabled:
                print("Biscuit can't be added")
        if print_enabled:
            print("\n")
           
    def remove_biscuit(self, biscuit, index, print_enabled=False):
        if print_enabled:
            print(f"Trying to remove (biscuit_{biscuit.id}, {index}):")
        if (biscuit,index) in self.biscuits_and_positions:
            self.biscuits_and_positions.remove((biscuit,index))
            
            #update score 
            self.score -= biscuit.value
            if print_enabled:
                print("Biscuit removed")
        else:
            if print_enabled:
                print("Biscuit not in roll")
        if print_enabled:
            print("\n")   
    
    def move_biscuit(self, biscuit, index, new_index, print_enabled=False):
        res = False
        self.remove_biscuit(biscuit, index, print_enabled=print_enabled)
        if not self.can_be_added(biscuit, new_index, print_enabled=print_enabled):
            self.add_biscuit(biscuit, index, print_enabled=print_enabled)
            if print_enabled:
                print("Biscuit can't be moved")
        else:
            self.add_biscuit(biscuit, new_index, print_enabled=print_enabled)
            res = True
            if print_enabled:
                print("Biscuit moved")
        return res
    
    def display_roll(self):
        
        print("Roll :")
        for biscuit in self.biscuits_and_positions:
            print(f"(biscuit_{biscuit[0].id}, {biscuit[1]})", end=", ")
        print("\n")
        roll = "-"*500
        
        for biscuit in self.biscuits_and_positions:
            list1 = list(roll)
            start_index = biscuit[1]
            end_index = biscuit[1]+biscuit[0].length          
            list1[start_index: end_index] = str(biscuit[0].id)*biscuit[0].length
            roll = "".join(list1)
   
        #le final print
        for i in range(0, len(roll), 100):
            print(roll[i:i+100], end="\n")
        print("_"*100+"\n", end="\n")
            
def display_roll_defects():
        defects_count = defects.groupby(['x', 'class']).size().unstack(fill_value=0)
        print(defects_count)
            
    
roll = Roll(biscuits_and_positions=[(biscuit_0,2)], length=500)
print(roll.defects, roll.score)

{'0': 'a', '1': 'a', '2': 'a', '3': 'b', '8': 'c', '9': 'c', '10': 'c', '12': 'a', '14': 'a', '16': 'a', '19': 'b', '21': 'b', '22': 'b', '23': 'a', '25': 'a', '26': 'a', '29': 'a', '30': 'b', '32': 'a', '33': 'b', '34': 'b', '35': 'b', '36': 'a', '37': 'b', '38': 'a', '39': 'c', '40': 'c', '41': 'c', '44': 'b', '46': 'a', '47': 'c', '48': 'b', '49': 'c', '50': 'c', '51': 'c', '53': 'a', '54': 'b', '55': 'a', '57': 'c', '58': 'a', '59': 'b', '64': 'c', '65': 'c', '68': 'b', '69': 'a', '72': 'c', '73': 'a', '74': 'c', '76': 'a', '77': 'a', '82': 'c', '83': 'a', '84': 'c', '85': 'a', '89': 'c', '91': 'c', '92': 'a', '93': 'a', '94': 'c', '95': 'c', '96': 'b', '97': 'c', '98': 'c', '102': 'c', '103': 'a', '107': 'c', '108': 'a', '110': 'c', '112': 'a', '113': 'c', '115': 'c', '116': 'a', '117': 'a', '118': 'a', '119': 'a', '120': 'c', '121': 'c', '122': 'b', '123': 'b', '124': 'a', '125': 'a', '127': 'b', '128': 'c', '129': 'c', '136': 'c', '137': 'c', '139': 'c', '140': 'a', '141': 'c', 

In [27]:
roll.add_biscuit(biscuit_1, 5, print_enabled=True)
roll.remove_biscuit(biscuit_1, 5, print_enabled=True)
roll.remove_biscuit(biscuit_0, 2, print_enabled=True)
roll.add_biscuit(biscuit_3, 495, print_enabled=True)
roll.add_biscuit(biscuit_1, 10, print_enabled=True)

Trying to add (biscuit_1, 5):
Biscuit overlap
Biscuit can't be added


Trying to remove (biscuit_1, 5):
Biscuit not in roll


Trying to remove (biscuit_0, 2):
Biscuit removed


Trying to add (biscuit_3, 495):
Biscuit can be added


Trying to add (biscuit_1, 10):
Biscuit can be added




In [28]:
roll.move_biscuit(biscuit_3, 495, 3, print_enabled=True)

Trying to remove (biscuit_3, 495):
Biscuit removed


Biscuit can be added
Trying to add (biscuit_3, 3):
Biscuit can be added


Biscuit moved


True

In [29]:
roll.display_roll()
display_defects()

Roll :
(biscuit_3, 3), (biscuit_1, 10), 

---33333--11111111----------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
____________________________________________________________________________________________________

Class A
2111--------1-1-1-----12-11--1--1---1-1-----1-1-1--1-1-1--11---------1---1--12-----1-2------12-1----
---1---11---1---1221----13--------------1--21---------1-1-11---3--1--1-11---------------311---------
---1-2------1-1-1-1----1---1--1-----------3---1----1-1----------------1----211-12---1---11----1-----
-11-1--------12--------------11-1-1-----

### Genetic algorithm implementation

In [30]:
#ici c'est en fait greedy search avec remplissage à gauche

population_size, biscuits = 1, [biscuit_0, biscuit_1, biscuit_2, biscuit_3]
population = []
biscuits.sort(key=lambda x: x.value/x.length, reverse=True)

for _ in range(population_size):
    roll = Roll(biscuits_and_positions=[], length=500)
    
    for biscuit in biscuits:
        for i in range(roll.length):
            if roll.can_be_added(biscuit, i):
                roll.add_biscuit(biscuit, i)
                
    population.append(roll)
    print("Score :", roll.score)
    population[_].display_roll()

Score : 705
Roll :
(biscuit_3, 2), (biscuit_2, 7), (biscuit_3, 10), (biscuit_3, 15), (biscuit_0, 20), (biscuit_3, 24), (biscuit_3, 29), (biscuit_2, 34), (biscuit_0, 36), (biscuit_3, 41), (biscuit_0, 46), (biscuit_3, 52), (biscuit_3, 58), (biscuit_3, 63), (biscuit_3, 68), (biscuit_0, 73), (biscuit_3, 77), (biscuit_3, 84), (biscuit_0, 89), (biscuit_0, 93), (biscuit_3, 99), (biscuit_3, 104), (biscuit_0, 109), (biscuit_0, 114), (biscuit_0, 118), (biscuit_0, 122), (biscuit_3, 126), (biscuit_3, 131), (biscuit_0, 136), (biscuit_0, 140), (biscuit_3, 144), (biscuit_3, 149), (biscuit_2, 154), (biscuit_3, 158), (biscuit_2, 164), (biscuit_3, 167), (biscuit_3, 172), (biscuit_3, 179), (biscuit_2, 185), (biscuit_3, 189), (biscuit_3, 195), (biscuit_3, 200), (biscuit_3, 205), (biscuit_3, 210), (biscuit_3, 215), (biscuit_3, 220), (biscuit_3, 225), (biscuit_3, 230), (biscuit_3, 235), (biscuit_0, 242), (biscuit_3, 247), (biscuit_3, 252), (biscuit_3, 257), (biscuit_3, 262), (biscuit_3, 267), (biscuit_0, 27

In [31]:
def initialize_population(population_size, biscuits):

    time_start = time.time()
    population = []
    biscuits.sort(key=lambda x: (x.value/x.length, x.length), reverse=True)

    for _ in range(population_size):
        roll = Roll(biscuits_and_positions=[], length=500)
        
        for biscuit in biscuits:
            fails = 0
            index = random.randint(0, roll.length-1)
            while fails < 50 : #arbitrary number (the more you increase the number the better the score but it scales poorly in time)
                if roll.can_be_added(biscuit, index):
                    roll.add_biscuit(biscuit, index)
                else:
                    fails += 1
                index = random.randint(0, roll.length-1)
                    
        population.append(roll)
        # print("Score :", roll.score)
        # population[_].display_roll()
    time_end = time.time()
    print("Average time per roll :", (time_end-time_start)/population_size)
    return population

population = initialize_population(10, [biscuit_0, biscuit_1, biscuit_2, biscuit_3])
population[0].display_roll()
print(population[0].score)
print(len(population))

Average time per roll : 0.5027999639511108
Roll :
(biscuit_2, 5), (biscuit_2, 12), (biscuit_0, 14), (biscuit_3, 18), (biscuit_1, 28), (biscuit_2, 37), (biscuit_3, 42), (biscuit_3, 52), (biscuit_3, 61), (biscuit_3, 67), (biscuit_3, 78), (biscuit_1, 83), (biscuit_0, 103), (biscuit_2, 123), (biscuit_1, 128), (biscuit_2, 137), (biscuit_2, 140), (biscuit_3, 144), (biscuit_3, 151), (biscuit_0, 161), (biscuit_0, 170), (biscuit_1, 175), (biscuit_0, 191), (biscuit_3, 195), (biscuit_0, 202), (biscuit_3, 209), (biscuit_1, 214), (biscuit_1, 229), (biscuit_1, 242), (biscuit_3, 252), (biscuit_2, 258), (biscuit_3, 261), (biscuit_3, 270), (biscuit_0, 280), (biscuit_3, 288), (biscuit_3, 294), (biscuit_1, 310), (biscuit_3, 324), (biscuit_0, 339), (biscuit_3, 345), (biscuit_1, 359), (biscuit_2, 370), (biscuit_0, 376), (biscuit_2, 380), (biscuit_1, 389), (biscuit_0, 399), (biscuit_3, 411), (biscuit_2, 418), (biscuit_2, 422), (biscuit_1, 425), (biscuit_0, 433), (biscuit_3, 439), (biscuit_0, 445), (biscuit_

In [32]:
def fitness_population(population):
    return [roll.score for roll in population]

fitness = fitness_population(population)
fitness

[422, 441, 414, 431, 420, 442, 437, 442, 433, 434]

In [33]:
def select_elitismRoulette(population, fitness, elitism_rate=0.1):
    population_with_fitness = list(zip(population, fitness))
    population_with_fitness.sort(key=lambda x: x[1], reverse=True)
    elites = [x[0] for x in population_with_fitness[:int(len(population_with_fitness)*elitism_rate)]]

    weights = [x[1] for x in population_with_fitness]
    selected = random.choices(population_with_fitness, weights=weights, k=len(population_with_fitness)-len(elites))
    return elites + [x[0] for x in selected]

def select_elitismRank(population, fitness, elitism_rate=0.1):
    population_with_fitness = list(zip(population, fitness))
    population_with_fitness.sort(key=lambda x: x[1], reverse=True)
    elites = [x[0] for x in population_with_fitness[:int(len(population_with_fitness)*elitism_rate)]]

    weights = [x for x in range(len(population_with_fitness),0,-1)]
    selected = random.choices(population_with_fitness, weights=weights, k=len(population_with_fitness)-len(elites))
    return elites + [x[0] for x in selected]

def select_tournament(population, fitness, tournament_size=3):
    selected = []
    for _ in range(len(population)):
        tournament = random.sample(list(zip(population, fitness)), tournament_size)
        winner = max(tournament, key=lambda x: x[1])
        selected.append(winner[0])
    return selected

#tests
selected1 = select_elitismRoulette(population, fitness)
print(fitness_population(selected1))

selected2 = select_elitismRank(population, fitness)
print(fitness_population(selected2))

selected3 = select_tournament(population, fitness)
print(fitness_population(selected3))

[442, 442, 437, 442, 434, 442, 434, 431, 420, 442]
[442, 442, 414, 442, 433, 442, 434, 442, 442, 414]
[441, 433, 442, 441, 437, 433, 433, 437, 442, 442]


In [34]:
#quelques problemes avec celle-ci, elle peut faire des trous
def crossover_one_point(roll1, roll2, print_enabled=False):
    child = Roll(biscuits_and_positions=[], length=500)
    
    crossover_point = random.randint(0, min(len(roll1.biscuits_and_positions), len(roll2.biscuits_and_positions)))
    if print_enabled:
        print(crossover_point, len(roll1.biscuits_and_positions), len(roll2.biscuits_and_positions))

    for i in range(crossover_point):
        child.add_biscuit(roll1.biscuits_and_positions[i][0], roll1.biscuits_and_positions[i][1])
    
    for i in range(crossover_point, len(roll2.biscuits_and_positions)):
        child.add_biscuit(roll2.biscuits_and_positions[i][0], roll2.biscuits_and_positions[i][1])
    
    return child

def crossover_uniform(roll1, roll2, print_enabled=False):
    child = Roll(biscuits_and_positions=[], length=500)
    
    index_roll1 = [x[1] for x in roll1.biscuits_and_positions]
    index_roll2 = [x[1] for x in roll2.biscuits_and_positions]
    merged_index = list(set(index_roll1 + index_roll2))
    merged_index.sort()
    if print_enabled:
        print(merged_index)

    for i in merged_index:
        if i in index_roll1 and i in index_roll2:
            if random.random() < 0.5:
                child.add_biscuit(roll1.biscuits_and_positions[index_roll1.index(i)][0], i)
            else:
                child.add_biscuit(roll2.biscuits_and_positions[index_roll2.index(i)][0], i)
        elif i in index_roll1:
            child.add_biscuit(roll1.biscuits_and_positions[index_roll1.index(i)][0], i)
        else:
            child.add_biscuit(roll2.biscuits_and_positions[index_roll2.index(i)][0], i)
    return child

#tests
child1 = crossover_one_point(selected1[0], selected1[1], print_enabled=True)
selected1[0].display_roll()
selected1[1].display_roll()
child1.display_roll()

child2 = crossover_uniform(selected1[0], selected1[1])
child2.display_roll()

56 61 61
Roll :
(biscuit_1, 8), (biscuit_3, 18), (biscuit_1, 25), (biscuit_2, 41), (biscuit_3, 45), (biscuit_0, 52), (biscuit_2, 65), (biscuit_3, 67), (biscuit_1, 75), (biscuit_3, 84), (biscuit_0, 90), (biscuit_2, 99), (biscuit_3, 102), (biscuit_0, 112), (biscuit_1, 120), (biscuit_3, 130), (biscuit_2, 138), (biscuit_1, 145), (biscuit_3, 167), (biscuit_2, 172), (biscuit_0, 175), (biscuit_0, 180), (biscuit_3, 190), (biscuit_3, 196), (biscuit_0, 201), (biscuit_3, 210), (biscuit_3, 217), (biscuit_1, 222), (biscuit_1, 232), (biscuit_3, 247), (biscuit_2, 254), (biscuit_3, 256), (biscuit_1, 263), (biscuit_3, 282), (biscuit_3, 287), (biscuit_0, 296), (biscuit_2, 306), (biscuit_0, 310), (biscuit_1, 315), (biscuit_3, 323), (biscuit_1, 330), (biscuit_2, 338), (biscuit_0, 342), (biscuit_3, 348), (biscuit_0, 356), (biscuit_0, 369), (biscuit_3, 376), (biscuit_0, 382), (biscuit_3, 389), (biscuit_1, 394), (biscuit_3, 411), (biscuit_0, 420), (biscuit_1, 425), (biscuit_1, 435), (biscuit_3, 448), (biscui

In [35]:
def mutation_add(roll, mutation_rate=0.1, print_enabled=False):
    
    roll_copy = Roll(biscuits_and_positions=[], length=500)
    for biscuit in roll.biscuits_and_positions.copy():
        roll_copy.add_biscuit(biscuit[0], biscuit[1])
    
    if print_enabled:
        print("Before mutation :")
        roll.display_roll()
    
    biscuits.sort( key=lambda x: x.value/x.length, reverse=True)

    #on enlève des biscuits au hasard
    for biscuit in roll.biscuits_and_positions:
        if random.random() < mutation_rate:
            roll.remove_biscuit(biscuit[0], biscuit[1])
            if print_enabled:
                print(f"Biscuit removed : (biscuit_{biscuit[0].id}, {biscuit[1]})")

    #on essaye d'ajouter un biscuit partout ou l'on peut 
    for i in range(roll.length):
        #if random.random() < mutation_rate:
        for biscuit in biscuits:
            if roll.can_be_added(biscuit, i):
                roll.add_biscuit(biscuit, i)
                if print_enabled:
                    print(f"Biscuit added : (biscuit_{biscuit.id}, {i})")
    
    if print_enabled:
        print("After mutation :")
        roll.display_roll()
   
    return [roll, roll_copy]


def mutation_swap(roll, mutation_rate=0.1, print_enabled=False):

    roll_copy = Roll(biscuits_and_positions=[], length=500)
    for biscuit in roll.biscuits_and_positions:
        roll_copy.add_biscuit(biscuit[0], biscuit[1])
    
    if print_enabled:
        print("Before mutation :")
        roll.display_roll()

    #on fait le swap de deux biscuits
    swap = False
    for biscuit in roll.biscuits_and_positions:
        if random.random() < mutation_rate:
            
            #on prend un biscuit différent au hasard
            biscuit1, index1 = roll.biscuits_and_positions[random.randint(0, len(roll.biscuits_and_positions)-1)]
            while biscuit1.id == biscuit[0].id:
                biscuit1, index1 = roll.biscuits_and_positions[random.randint(0, len(roll.biscuits_and_positions)-1)]
            
            roll.remove_biscuit(biscuit1, index1)
            roll.remove_biscuit(biscuit[0], biscuit[1])

            if print_enabled:
                print(f"(biscuit_{biscuit[0].id}, {biscuit[1]})")
                print(f"(biscuit_{biscuit1.id}, {index1})")

            if roll.can_be_added(biscuit[0], index1) and roll.can_be_added(biscuit1, biscuit[1]):
                roll.add_biscuit(biscuit[0], index1)
                roll.add_biscuit(biscuit1, biscuit[1])
                swap = True
                if print_enabled:
                    print("Biscuits swapped")
            else:
                roll.add_biscuit(biscuit[0], biscuit[1])
                roll.add_biscuit(biscuit1, index1)
                if print_enabled:
                    print("Biscuits not swapped")
    #on gere le cas ou le swap n'a pas marché
    # ....
    if print_enabled:
        print("After mutation :")
        roll.display_roll()
   
    return [roll, roll_copy]


def mutation_move(roll, mutation_rate=0.1, print_enabled=False):

    roll_copy = roll
    if print_enabled:
        print("Before mutation :")
        roll_copy.display_roll()

    #on essaye de déplacer un biscuit
    for biscuit in roll.biscuits_and_positions:
        if random.random() < mutation_rate:
            move_direction = random.choice([-1,1])
            index = biscuit[1] + move_direction

            #on verifie les limites du roll
            if index >= 0 and index+biscuit[0].length-1 <= roll.length-1:
                roll.remove_biscuit(biscuit[0], biscuit[1])
                if roll.can_be_added(biscuit[0], index):
                    roll.add_biscuit(biscuit[0], index)
                    if print_enabled:
                        print(f"Biscuit moved : (biscuit_{biscuit[0].id}, {biscuit[1]}) -> (biscuit_{biscuit[0].id}, {index})")
                        print("Biscuit moved")
                else:
                    roll.add_biscuit(biscuit[0], biscuit[1])
                    if print_enabled:
                        print("Biscuit not moved")
    if print_enabled:
        print("After mutation :")
        roll.display_roll()
    
    return [roll, roll_copy]
    

#tests
child1.remove_biscuit(biscuit_2, 24)
mutation_added = mutation_add(child1, print_enabled=True)
print(mutation_added[0].score, mutation_added[1].score)
mutation_swapped = mutation_swap(child1, print_enabled=True)
print(mutation_swapped[0].score, mutation_swapped[1].score)
mutation_moved = mutation_move(child1, print_enabled=True)
print(mutation_moved[0].score, mutation_moved[1].score)

Before mutation :
Roll :
(biscuit_1, 8), (biscuit_3, 18), (biscuit_1, 25), (biscuit_2, 41), (biscuit_3, 45), (biscuit_0, 52), (biscuit_2, 65), (biscuit_3, 67), (biscuit_1, 75), (biscuit_3, 84), (biscuit_0, 90), (biscuit_2, 99), (biscuit_3, 102), (biscuit_0, 112), (biscuit_1, 120), (biscuit_3, 130), (biscuit_2, 138), (biscuit_1, 145), (biscuit_3, 167), (biscuit_2, 172), (biscuit_0, 175), (biscuit_0, 180), (biscuit_3, 190), (biscuit_3, 196), (biscuit_0, 201), (biscuit_3, 210), (biscuit_3, 217), (biscuit_1, 222), (biscuit_1, 232), (biscuit_3, 247), (biscuit_2, 254), (biscuit_3, 256), (biscuit_1, 263), (biscuit_3, 282), (biscuit_3, 287), (biscuit_0, 296), (biscuit_2, 306), (biscuit_0, 310), (biscuit_1, 315), (biscuit_3, 323), (biscuit_1, 330), (biscuit_2, 338), (biscuit_0, 342), (biscuit_3, 348), (biscuit_0, 356), (biscuit_0, 369), (biscuit_3, 376), (biscuit_0, 382), (biscuit_3, 389), (biscuit_1, 394), (biscuit_3, 411), (biscuit_0, 420), (biscuit_1, 425), (biscuit_1, 435), (biscuit_3, 448)

In [36]:
def mutation_range(roll, mutation_rate=0.1, print_enabled=False):

    biscuits.sort( key=lambda x: x.value/x.length, reverse=True)
    roll.biscuits_and_positions.sort(key=lambda x: x[1])
    
    roll_copy = Roll(biscuits_and_positions=[], length=500)
    for biscuit in roll.biscuits_and_positions.copy():
        roll_copy.add_biscuit(biscuit[0], biscuit[1])
    
    if print_enabled:
        print("Before mutation :")
        roll.display_roll()

    a = random.randint(0, len(roll.biscuits_and_positions)-1)
    b = random.randint(0, len(roll.biscuits_and_positions)-1)
    while a == b or (roll.biscuits_and_positions[max(a,b)][1] - roll.biscuits_and_positions[min(a,b)][1] > 200):
        a = random.randint(0, len(roll.biscuits_and_positions)-1)
        b = random.randint(0, len(roll.biscuits_and_positions)-1)
    
    
    #on supprime les biscuits dans la range
    if print_enabled:
        print(f"Biscuit removed between {roll.biscuits_and_positions[min(a,b)][1]} and {roll.biscuits_and_positions[max(a,b)][1]}")
    for i in range(min(a,b), max(a,b)+1):
        roll.remove_biscuit(roll_copy.biscuits_and_positions[i][0], roll_copy.biscuits_and_positions[i][1])
    

    if print_enabled:
        print("While mutation :")
        roll.display_roll()
    
    #on essaye d'ajouter un biscuit partout ou l'on peut
    for i in range(roll_copy.biscuits_and_positions[min(a,b)][1], roll_copy.biscuits_and_positions[max(a,b)][1]):
        #if random.random() < mutation_rate:
        for biscuit in biscuits:
            if roll.can_be_added(biscuit, i):
                roll.add_biscuit(biscuit, i)
                if print_enabled:
                    print(f"Biscuit added : (biscuit_{biscuit.id}, {i})")
    
    if print_enabled:
        print("After mutation :")
        roll.display_roll()
   
    return [roll, roll_copy]

#tests
mutation_ranged = mutation_range(child1, print_enabled=True)

Before mutation :
Roll :
(biscuit_1, 0), (biscuit_1, 8), (biscuit_2, 16), (biscuit_3, 18), (biscuit_1, 25), (biscuit_2, 33), (biscuit_2, 35), (biscuit_2, 37), (biscuit_2, 41), (biscuit_2, 43), (biscuit_3, 45), (biscuit_0, 52), (biscuit_0, 56), (biscuit_3, 60), (biscuit_2, 65), (biscuit_3, 67), (biscuit_2, 72), (biscuit_1, 75), (biscuit_3, 84), (biscuit_0, 90), (biscuit_2, 95), (biscuit_0, 98), (biscuit_3, 102), (biscuit_0, 107), (biscuit_0, 112), (biscuit_1, 120), (biscuit_3, 130), (biscuit_2, 135), (biscuit_2, 138), (biscuit_0, 140), (biscuit_1, 145), (biscuit_2, 153), (biscuit_2, 155), (biscuit_3, 158), (biscuit_2, 164), (biscuit_3, 167), (biscuit_2, 172), (biscuit_0, 175), (biscuit_0, 180), (biscuit_2, 185), (biscuit_3, 190), (biscuit_3, 196), (biscuit_0, 201), (biscuit_3, 205), (biscuit_3, 210), (biscuit_2, 215), (biscuit_3, 217), (biscuit_1, 222), (biscuit_2, 230), (biscuit_1, 232), (biscuit_0, 242), (biscuit_3, 247), (biscuit_2, 252), (biscuit_2, 254), (biscuit_3, 256), (biscuit_

In [37]:
def find_holes(roll):
    roll.biscuits_and_positions.sort(key=lambda x: x[1])
    holes = []
    
    #find begining holes and end holes
    if roll.biscuits_and_positions[0][1] > 0:
        holes.append((0, roll.biscuits_and_positions[0][1]-1))
    if roll.biscuits_and_positions[-1][1] < roll.length-1:
        holes.append((roll.biscuits_and_positions[-1][1]+roll.biscuits_and_positions[-1][0].length, roll.length-1))

    #find holes between biscuits
    for i in range(len(roll.biscuits_and_positions)-1):
        if roll.biscuits_and_positions[i+1][1] - roll.biscuits_and_positions[i][1] > roll.biscuits_and_positions[i][0].length:
            holes.append((roll.biscuits_and_positions[i][1]+roll.biscuits_and_positions[i][0].length, roll.biscuits_and_positions[i+1][1]-1))
    return holes


def mutation_holes(roll, mutation_rate=0.1, print_enabled=False):

    roll_copy = Roll(biscuits_and_positions=[], length=500)
    for biscuit in roll.biscuits_and_positions.copy():
        roll_copy.add_biscuit(biscuit[0], biscuit[1])
    
    if print_enabled:
        print("Before mutation :")
        roll.display_roll()

    holes = find_holes(roll)
    if print_enabled:
        print("Holes :", holes)
    selected_holes = []
    for hole in holes:
        if random.random() < mutation_rate:
            selected_holes.append(hole)
    
    if print_enabled:
        print("Selected holes :", selected_holes)
    #we extend the holes of length at most 20
    a = random.randint(5, 15)
    for hole in selected_holes:
        if hole[0]-a >= 0:
            hole = (hole[0]-a, hole[1])
        else:
            hole = (0, hole[1])
        if hole[1]+a <= roll.length-1:
            hole = (hole[0], hole[1]+a)
        else:
            hole = (hole[0], roll.length-1)
        
        #we remove the biscuits in the range
        for i in range(hole[0], hole[1]+1):
            for biscuit in biscuits:
                roll.remove_biscuit(biscuit, i)

        #we try to add a biscuit everywhere we can
        biscuits_copy = biscuits.copy()
        random_biscuits = []
        for biscuit in biscuits:
            selected_biscuit = random.choices(biscuits_copy, weights=[x.value/x.length for x in biscuits_copy], k=1)[0]
            random_biscuits.append(selected_biscuit)
            biscuits_copy.remove(selected_biscuit)

        for i in range(hole[0], hole[1]+1):
            for biscuit in random_biscuits:
                if roll.can_be_added(biscuit, i):
                    roll.add_biscuit(biscuit, i)
                    if print_enabled:
                        print(f"Biscuit added : (biscuit_{biscuit.id}, {i})")
    
    if print_enabled:
        print("After mutation :")
        roll.display_roll()
    
    return [roll, roll_copy]

#tests
mutation_holed = mutation_holes(child1, print_enabled=True)
print(mutation_holed[0].score, mutation_holed[1].score)

Before mutation :
Roll :
(biscuit_1, 0), (biscuit_1, 8), (biscuit_2, 16), (biscuit_3, 18), (biscuit_1, 25), (biscuit_2, 33), (biscuit_2, 35), (biscuit_2, 37), (biscuit_2, 41), (biscuit_2, 43), (biscuit_3, 45), (biscuit_0, 52), (biscuit_0, 56), (biscuit_3, 60), (biscuit_2, 65), (biscuit_3, 67), (biscuit_2, 72), (biscuit_1, 75), (biscuit_3, 84), (biscuit_0, 90), (biscuit_2, 95), (biscuit_0, 98), (biscuit_3, 102), (biscuit_0, 107), (biscuit_0, 112), (biscuit_1, 120), (biscuit_3, 130), (biscuit_2, 135), (biscuit_2, 138), (biscuit_0, 140), (biscuit_1, 145), (biscuit_2, 153), (biscuit_2, 155), (biscuit_3, 158), (biscuit_2, 164), (biscuit_3, 167), (biscuit_2, 172), (biscuit_0, 175), (biscuit_0, 180), (biscuit_2, 185), (biscuit_3, 190), (biscuit_3, 196), (biscuit_0, 201), (biscuit_3, 205), (biscuit_3, 210), (biscuit_2, 215), (biscuit_3, 217), (biscuit_1, 222), (biscuit_2, 230), (biscuit_1, 232), (biscuit_0, 242), (biscuit_3, 247), (biscuit_2, 252), (biscuit_2, 254), (biscuit_3, 256), (biscuit_

In [38]:
def genetic_algorithm(population_size=10, generations=1000, elite_rate=0.1, mutation_rate=0.1):
    time_start = time.time()
    #Initialisation de la population
    population = initialize_population(population_size, [biscuit_0, biscuit_1, biscuit_2, biscuit_3])
    best_indiv = max(population, key=lambda x: x.score)
    best_indiv_score = best_indiv.score
    fit_list = []
    no_improvement_streak = 0
    print("_"*100+"\n", end="\n")
    
    for _ in range(generations):

        time_start_gen = time.time()
        new_population = []
        parents = select_elitismRoulette(population, fitness_population(population), elitism_rate=elite_rate)
        for i in range(0,len(parents),2):
            child1, child2 = crossover_uniform(parents[i], parents[i+1]), crossover_uniform(parents[i], parents[i+1])
            
            child1 = mutation_range(child1, mutation_rate=mutation_rate)[0]
            child2 = mutation_range(child2, mutation_rate=mutation_rate)[0]
            child1 = mutation_move(child1, mutation_rate=mutation_rate)[0]
            child2 = mutation_move(child2, mutation_rate=mutation_rate)[0]
            child1 = mutation_holes(child1, mutation_rate=mutation_rate)[0]
            child2 = mutation_holes(child2, mutation_rate=mutation_rate)[0]
            
            new_population.append(child1)
            new_population.append(child2)

        population = new_population
        # print(len(new_population))
        # for i in new_population:
        #     print(i.score)
        current_best_indiv = max(population, key=lambda x: x.score)
        current_best_indiv_score = current_best_indiv.score

        if current_best_indiv_score > best_indiv_score:
            best_indiv = current_best_indiv
            best_indiv_score = current_best_indiv_score
            no_improvement_streak = 0
        else:
            no_improvement_streak += 1
            if no_improvement_streak > 100:
                break

        fit_list.append(best_indiv_score)
        time_end_gen = time.time()
        
        print("Generation :", _)
        print("Best score :", best_indiv_score)
        print("Average score :", sum(fitness_population(population))/len(fitness_population(population)))
        print("Worst score :", min(fitness_population(population)))
        print("Time for generation :", (time_end_gen-time_start_gen))
        print("_"*100+"\n", end="\n")
    time_end = time.time()
    print("Average time per generation :", (time_end-time_start)/generations)
    return best_indiv, best_indiv_score, fit_list

best_indiv, best_indiv_score, fit_list = genetic_algorithm(population_size=40, generations=1000, elite_rate=0.1, mutation_rate=0.3)
        
        

Average time per roll : 0.43769699335098267
____________________________________________________________________________________________________

Generation : 0
Best score : 683
Average score : 622.275
Worst score : 571
Time for generation : 98.29879212379456
____________________________________________________________________________________________________

Generation : 1
Best score : 709
Average score : 684.475
Worst score : 621
Time for generation : 74.642507314682
____________________________________________________________________________________________________

Generation : 2
Best score : 729
Average score : 699.15
Worst score : 661
Time for generation : 71.10961294174194
____________________________________________________________________________________________________

Generation : 3
Best score : 730
Average score : 707.225
Worst score : 678
Time for generation : 65.24443674087524
_______________________________________________________________________________________________

In [39]:
best_indiv.display_roll()

Roll :
(biscuit_1, 0), (biscuit_1, 8), (biscuit_3, 16), (biscuit_0, 21), (biscuit_0, 25), (biscuit_3, 29), (biscuit_2, 34), (biscuit_0, 36), (biscuit_0, 40), (biscuit_0, 44), (biscuit_2, 48), (biscuit_0, 50), (biscuit_0, 54), (biscuit_0, 58), (biscuit_3, 62), (biscuit_3, 67), (biscuit_0, 72), (biscuit_0, 76), (biscuit_1, 80), (biscuit_3, 88), (biscuit_0, 93), (biscuit_1, 98), (biscuit_2, 106), (biscuit_3, 108), (biscuit_0, 114), (biscuit_0, 118), (biscuit_0, 122), (biscuit_3, 126), (biscuit_3, 131), (biscuit_0, 136), (biscuit_0, 140), (biscuit_3, 144), (biscuit_3, 149), (biscuit_2, 154), (biscuit_0, 158), (biscuit_0, 162), (biscuit_0, 167), (biscuit_0, 171), (biscuit_0, 175), (biscuit_0, 179), (biscuit_2, 185), (biscuit_1, 187), (biscuit_3, 195), (biscuit_3, 200), (biscuit_3, 205), (biscuit_3, 210), (biscuit_3, 215), (biscuit_3, 220), (biscuit_1, 225), (biscuit_1, 233), (biscuit_1, 242), (biscuit_3, 250), (biscuit_3, 255), (biscuit_3, 260), (biscuit_3, 265), (biscuit_3, 270), (biscuit_

In [40]:
len(best_indiv.biscuits_and_positions)

101